This file serves as the beginning towards my research showing the not-applicablility of Yuanfang method to this drugcombdb.
1) See to what extent I can rebuild their model
2) with reduced data from drugcombdb
3) applied to 1 drugcombdb source
4) applied to extended sources<br>
<br>If I have time later, I can consider trying to implement this mouse-gene method or look at their cell representations

Central claim: The main outcome of Yuanfang is purely owed to the monotherapy predictor

In [1]:
import numpy as np
import pandas as pd

This is the "Quality Control" information, an inappropriate name imo

In [4]:
data = pd.read_csv("./ch1_train_combination_and_monoTherapy.csv")
print(data.shape)
data.iloc[12:28]

(2199, 14)


,CELL_LINE,COMPOUND_A,COMPOUND_B,MAX_CONC_A,MAX_CONC_B,IC50_A,H_A,Einf_A,IC50_B,H_B,Einf_B,SYNERGY_SCORE,QA,COMBINATION_ID
12,MDA-MB-453,AKT,ADAM17,1.0,75.0,0.282664,2.263455,34.219723,1.000000,0.000000,100.000000,16.049800,1,ADAM17.AKT
13,M14,AKT,ADAM17,1.0,75.0,0.276412,3.749313,36.292590,0.007500,0.100000,83.937053,30.526590,1,ADAM17.AKT
14,BT-549,BCL2_BCL2L1,ADAM17,1.0,75.0,0.000100,0.100000,79.232718,0.007500,0.100000,90.995525,-7.039191,1,ADAM17.BCL2_BCL2L1
15,CAL-148,BCL2_BCL2L1,ADAM17,1.0,75.0,0.288908,3.871862,35.479731,1.000000,0.000000,100.000000,-17.522800,-1,ADAM17.BCL2_BCL2L1
16,CAMA-1,BCL2_BCL2L1,ADAM17,1.0,75.0,1.000000,0.351607,65.814282,0.007500,0.100000,92.508112,3.764896,1,ADAM17.BCL2_BCL2L1
17,DU-4475,BCL2_BCL2L1,ADAM17,1.0,75.0,0.008057,10.000000,0.074954,37.436064,1.372342,30.854525,3.335391,1,ADAM17.BCL2_BCL2L1
18,HCC1395,BCL2_BCL2L1,ADAM17,1.0,75.0,1.000000,0.141016,86.381581,75.000000,10.000000,94.600237,-1.784113,1,ADAM17.BCL2_BCL2L1
19,HCC1806,BCL2_BCL2L1,ADAM17,1.0,75.0,1.000000,0.000000,100.000000,1.000000,0.000000,100.000000,2.537269,1,ADAM17.BCL2_BCL2L1
20,HCC70,BCL2_BCL2L1,ADAM17,1.0,75.0,1.000000,0.000000,100.000000,1.000000,0.000000,100.000000,-12.643010,1,ADAM17.BCL2_BCL2L1
21,MDA-MB-157,BCL2_BCL2L1,ADAM17,1.0,75.0,0.333916,0.691399,76.917447,0.007500,4.234800,90.900000,-2.891425,1,ADAM17.BCL2_BCL2L1


Why are there only 2199 records? I'd lowkey expect 11k

In [15]:
data[data["CELL_LINE"].str.lower()=="vcap"]

,CELL_LINE,COMPOUND_A,COMPOUND_B,MAX_CONC_A,MAX_CONC_B,IC50_A,H_A,Einf_A,IC50_B,H_B,Einf_B,SYNERGY_SCORE,QA,COMBINATION_ID
125,VCaP,ERBB,AKT,3.0,1.0,0.814719,2.565062,77.342449,0.437597,1.396308,57.875531,5.142114,1,AKT.ERBB
1905,VCaP,PIK3CB_PIK3CD,PIK3CB_PIK3CD,1.0,1.0,1.000000,0.225404,56.634046,0.010625,10.000000,0.000000,-128.049100,-3,PIK3CB_PIK3CD.PIK3CB_PIK3CD


Next: Get Raw Data (appentrly I can't copy them from Drugcombdb bc they don't seem to exist there)

In [35]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = './ch1_training'
csv_files = glob.glob(path + "/*.csv")

In [64]:
df_list = (pd.DataFrame(np.array(pd.read_csv(file,header=None)).flatten()).transpose() for file in csv_files)
df   = pd.concat(df_list, ignore_index=True)

In [65]:
#very ugly containing all information
df

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
0,NaN,0,0.03,0.1,0.3,1.0,3.0,(=Agent 2),0,100,...,NaN,NaN,Title,SW900,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0,0.03,0.1,0.3,1.0,3.0,(=Agent 2),0,100,...,NaN,NaN,Title,NCI-H838,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0,0.03,0.1,0.3,1.0,3.0,(=Agent 2),0,100,...,NaN,NaN,Title,NCI-H522,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0,0.03,0.1,0.3,1.0,3.0,(=Agent 2),0,100,...,NaN,NaN,Title,NCI-H520,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0,0.03,0.1,0.3,1.0,3.0,(=Agent 2),0,100,...,NaN,NaN,Title,NCI-H358,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,NaN,0,0.75,2.5,7.5,25.0,75.0,(=Agent 2),0,100,...,NaN,NaN,Title,HCC1143,NaN,NaN,NaN,NaN,NaN,NaN
2195,NaN,0,0.75,2.5,7.5,25.0,75.0,(=Agent 2),0,100,...,NaN,NaN,Title,DU-4475,NaN,NaN,NaN,NaN,NaN,NaN
2196,NaN,0,0.75,2.5,7.5,25.0,75.0,(=Agent 2),0,100,...,NaN,NaN,Title,CAL-51,NaN,NaN,NaN,NaN,NaN,NaN
2197,NaN,0,0.75,2.5,7.5,25.0,75.0,(=Agent 2),0,100,...,NaN,NaN,Title,CAL-120,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
everything in loc columns
1. col 65 is Agent1
2. col 73 Agent2
3. columns 1:7 conc Agent 2
4. columns [8,16,24,32,40,48] conc Agent 1
5. Monotherapy values +1 for agent1, +8 on both ends of the array for agent2

In [89]:
df.loc[:,[8,16,24,32,40,48]]

,8,16,24,32,40,48,56,64
0,0,0.03,0.1,0.3,1,3,(=Agent 1),Agent1
1,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1
2,0,0.03,0.1,0.3,1,3,(=Agent 1),Agent1
3,0,0.03,0.1,0.3,1,3,(=Agent 1),Agent1
4,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1
...,...,...,...,...,...,...,...,...
2194,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1
2195,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1
2196,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1
2197,0,0.01,0.03,0.1,0.3,1,(=Agent 1),Agent1


In [10]:
data[data["COMPOUND_A"]=="Oxaliplatin"]

,CELL_LINE,COMPOUND_A,COMPOUND_B,MAX_CONC_A,MAX_CONC_B,IC50_A,H_A,Einf_A,IC50_B,H_B,Einf_B,SYNERGY_SCORE,QA,COMBINATION_ID
407,COLO-205,Oxaliplatin,BCL2,10.0,3.0,0.376569,1.228914,0.000000,0.291322,2.091344,2.951622,-4.177698,1,BCL2.Oxaliplatin
408,COLO-205,Oxaliplatin,BCL2,1.0,1.0,0.154431,1.056109,36.701972,0.356587,2.505795,0.000000,-8.298241,1,BCL2.Oxaliplatin
409,HCT-116,Oxaliplatin,BCL2,10.0,1.0,8.651186,10.000000,0.000000,0.069475,0.834797,46.228114,-186.879400,-1,BCL2.Oxaliplatin
410,HCT-116,Oxaliplatin,BCL2,1.0,3.0,0.285753,1.772688,4.369306,1.833796,1.348885,0.000000,45.091200,1,BCL2.Oxaliplatin
411,RKO,Oxaliplatin,BCL2,3.0,3.0,1.317725,2.876552,0.000000,3.000000,1.619496,40.303683,-3.900104,1,BCL2.Oxaliplatin
412,RKO,Oxaliplatin,BCL2,3.0,3.0,2.782220,10.000000,0.000000,0.028861,1.471660,86.925311,-88.019040,-1,BCL2.Oxaliplatin
413,SW48,Oxaliplatin,BCL2,3.0,3.0,0.175281,0.904967,0.000000,0.179331,1.111829,20.059491,59.118200,1,BCL2.Oxaliplatin
414,SW48,Oxaliplatin,BCL2,0.3,3.0,0.036607,10.000000,32.598831,0.633250,1.304931,41.418723,33.720600,1,BCL2.Oxaliplatin
415,SW620,Oxaliplatin,BCL2,10.0,0.3,0.408438,1.264366,2.154965,0.300000,0.195173,85.761916,-1.120614,1,BCL2.Oxaliplatin
416,SW620,Oxaliplatin,BCL2,3.0,3.0,0.105606,1.984775,10.371550,1.000000,0.000000,100.000000,13.351560,-1,BCL2.Oxaliplatin


In [9]:
dta[dta["Drug1"]=="Oxaliplatin"]

,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
481253,481254,Oxaliplatin,PF-431396,A-673,0.508,1.281,-7.917,1.716
487117,487118,Oxaliplatin,Ruxolitinib,L-1236,11.652,7.790,-21.555,-4.571
494109,494110,Oxaliplatin,Marizomib / Salinosporamide A,DIPG25,3.612,1.426,-30.600,-1.153
496559,496560,Oxaliplatin,Panobinostat,DIPG25,2.793,4.313,-3.793,3.297


In [8]:
set(data["COMPOUND_A"]).intersection(set(dta["Drug1"]))

{'Chloroquine', 'Cisplatin', 'Gemcitabine', 'Oxaliplatin', 'Vinorelbine'}

In [5]:
import pandas as pd
dta = pd.read_csv("../drugCombDBdata/synergy-score-data/REGRdrugcombs_scored.csv")
dta.head()

,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
0,1,5-FU,ABT-888,A2058,1.72,6.26,-2.75,5.54
1,2,5-FU,ABT-888,A2058,5.88,12.33,3.33,11.61
2,3,5-FU,ABT-888,A2058,3.59,11.66,2.65,10.94
3,4,5-FU,ABT-888,A2058,-0.85,5.15,-3.86,4.43
4,5,5-FU,AZD1775,A2058,12.29,15.77,10.40,18.66


In [6]:
dta[dta["Cell line"]=="A549"]

,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
132420,132421,IMATINIB,CARMUSTINE,A549,0.29,2.65,-0.81,0.45
132442,132443,PACLITAXEL,ZOLEDRONIC ACID,A549,1.56,7.82,-31.31,4.61
132606,132607,CARBOPLATIN,HYDROXYUREA,A549,-7.41,-2.31,-10.45,-0.97
132675,132676,NSC-127716,AMIFOSTINE,A549,-0.72,-2.00,-2.93,-2.82
132689,132690,SORAFENIB,ERLOTINIB HYDROCHLORIDE,A549,2.46,1.25,-5.48,-1.77
...,...,...,...,...,...,...,...,...
443849,443850,CRIZOTINIB,ADM HYDROCHLORIDE,A549,-3.85,-7.20,-13.05,-4.55
443855,443856,ROMIDEPSIN,ADM HYDROCHLORIDE,A549,0.27,-3.01,1.42,2.09
443859,443860,PRALATREXATE,ADM HYDROCHLORIDE,A549,-6.78,-9.04,-1.91,-1.11
443867,443868,AXITINIB,ADM HYDROCHLORIDE,A549,-0.10,-0.24,-2.26,1.58
